In [50]:
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
import torch
from torch import optim, nn
import os
import sys
from PIL import Image, ImageOps
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt
from os import path
import urllib.request
import pdb
from torch.utils import data

root = '/content/dataset'
train_dataset = torchvision.datasets.CIFAR10(root, train=True, transform=None, target_transform=None, download=True)
test_dataset = torchvision.datasets.CIFAR10(root, train=False, transform=None, target_transform=None, download=True)

class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self,dataset):
    'Initialization'
    self.dataset=dataset
  def __len__(self):
    'Denotes the total number of samples'
    return len(self.dataset)
    #return 1024
  def __getitem__(self, index):
    'Generates one sample of data'
    # Select sample
    x,label = (self.dataset[index])

    transform=transforms.Compose([transforms.ToTensor()])
    im = transform(x)
    return im, label

train_loader = Dataset(train_dataset)
test_loader = Dataset(test_dataset)

batch_size=20
params={'batch_size':batch_size,'shuffle':True}
training_generator=data.DataLoader(train_loader,**params)
test_generator=data.DataLoader(test_loader,**params)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
class resnet20(nn.Module):
  def __init__(self):
    super(resnet20,self).__init__()
    c=3
    out = 10
    channels=32
    #####Create the first block
    self.conv1=nn.Conv2d(c,channels,(3,3),padding=(1,1))
    self.conv2=nn.Conv2d(channels,channels,(3,3),padding=(1,1))

    self.conv8=nn.Conv2d(channels,channels*2,(3,3),padding=(1,1))
    self.conv9=nn.Conv2d(channels*2,channels*2,(3,3),padding=(1,1))
    
    self.conv15=nn.Conv2d(channels*4,channels*4,(3,3),padding=(1,1))
    self.conv16=nn.Conv2d(channels*4,channels*4,(3,3),padding=(1,1))
    
    self.bn1=nn.BatchNorm2d(channels)
    self.activation = nn.ReLU()


  
  def forward(self,x):
    def block_1(in1):
      o2=self.activation(self.bn1(self.conv2(in1)))
      o3=self.conv2(o2)
      o4=in1 + o3
      o5 = self.activation(self.bn1(o4))
      return(o5)

    n,c,h,w = x.size()
    #3 channels
    o1=self.activation(self.bn1(self.conv1(x)))
    #32 channels
    pdb.set_trace()
    block1=block_1(o1)
    block2=block_1(block1)
    block3=block_1(block2)
    #32 channels

    block4
    block5

    #64 channels
    
    block6
    block7
    block8

    #64 channels

    block9
    block10

    #128 channels

    block11
    block12
    block13

    #128 channels

    block14
    block15

    #256 channels

    block16
    block17
    block18
    block19
    block20

    #256 channels

    fc 

    return o27.squeeze(2).squeeze(2)

model = resnet20()

In [86]:
for image,label in training_generator:
  yhat = model(image)
  break

> <ipython-input-85-9332c1b39008>(34)forward()
-> block1=block(o1)
(Pdb) n
> <ipython-input-85-9332c1b39008>(35)forward()
-> block2=block(block1)
(Pdb) block1.size()
torch.Size([20, 32, 32, 32])
(Pdb) n
> <ipython-input-85-9332c1b39008>(36)forward()
-> block3=block(block2)
(Pdb) block2.size()
torch.Size([20, 32, 32, 32])
(Pdb) n
> <ipython-input-85-9332c1b39008>(39)forward()
-> return o27.squeeze(2).squeeze(2)
(Pdb) block3.size()
torch.Size([20, 32, 32, 32])
(Pdb) 



BdbQuit: ignored